## Load the digits dataset and evolutionary_search

In [2]:
from evolutionary_search import EvolutionaryAlgorithmSearchCV
import sklearn.datasets
import numpy as np
import pandas as pd

data = sklearn.datasets.load_digits()
X = data["data"]
y = data["target"]

# make it a 2-class problem by only classifying the digit "5" vs the rest
y = np.array([1 if yy == 5 else 0 for yy in y])

X.shape, y.shape

((1797, 64), (1797,))

In [3]:
from sklearn.model_selection import StratifiedKFold, GridSearchCV, RandomizedSearchCV
from sklearn.svm import SVC

## Train an SVM with RBF kernel

### Using conventional GridSearchCV

Parameter grid: 625 parameter combinations

In [5]:
paramgrid = {"kernel": ["rbf"],
             "C"     : np.logspace(-9, 9, num=25, base=10),
             "gamma" : np.logspace(-9, 9, num=25, base=10)}
print("Size: ", len(paramgrid["kernel"])*len(paramgrid["C"])*len(paramgrid["gamma"]))

Size:  625


In [4]:
cv = GridSearchCV(estimator=SVC(),
                  param_grid=paramgrid,
                  scoring="accuracy",
                  cv=StratifiedKFold(n_splits=2),
                  verbose=1)
%time cv.fit(X, y)

Fitting 2 folds for each of 625 candidates, totalling 1250 fits
Wall time: 3min 57s


[Parallel(n_jobs=1)]: Done 1250 out of 1250 | elapsed:  4.0min finished


GridSearchCV(cv=StratifiedKFold(n_splits=2, random_state=None, shuffle=False),
       error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'gamma': array([  1.00000e-09,   5.62341e-09,   3.16228e-08,   1.77828e-07,
         1.00000e-06,   5.62341e-06,   3.16228e-05,   1.77828e-04,
         1.00000e-03,   5.62341e-03,   3.16228e-02,   1.77828e-01,
         1.00000e+00,   5.62341e+00,   3.16228e+01,   1.77828e+02,
         1....7828e+05,
         1.00000e+06,   5.62341e+06,   3.16228e+07,   1.77828e+08,
         1.00000e+09])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='accuracy', verbose=1)

### Best score + params

In [10]:
cv.best_score_, cv.best_params_

(0.98942682248191427, {'C': 1.0, 'gamma': 0.001, 'kernel': 'rbf'})

An example of the "cannonical" cv_results_ table in sklearn:

In [34]:
pd.DataFrame(cv.cv_results_).sort_values("mean_test_score", ascending=False).head()

index  max_test_score  mean_test_score  min_test_score nan_test_score?  \
0        1        0.994435         0.994435        0.994435           False   
254    255        0.994435         0.994435        0.994435           False   
263    264        0.994435         0.994435        0.994435           False   
262    263        0.994435         0.994435        0.994435           False   
261    262        0.994435         0.994435        0.994435           False   

     param_index                                             params  \
0              0  {'C': 177827941.004, 'kernel': 'rbf', 'gamma':...   
254            0  {'C': 177827.941004, 'kernel': 'rbf', 'gamma':...   
263            0  {'C': 177827941.004, 'kernel': 'rbf', 'gamma':...   
262            0  {'C': 1000000.0, 'kernel': 'rbf', 'gamma': 0.001}   
261            0  {'C': 177827941.004, 'kernel': 'rbf', 'gamma':...   

     std_test_score  
0               0.0  
254             0.0  
263             0.0  
262             0.0  
261             0.0

### Using RandomizedSearchCV

Same parameter space, but only test 250 random combinations.

In [12]:
cv = RandomizedSearchCV(estimator=SVC(),
                        param_distributions=paramgrid,
                        n_iter=250,
                        scoring="accuracy",
                        cv=StratifiedKFold(n_splits=2),
                        verbose=1)
%time cv.fit(X, y)

Fitting 2 folds for each of 250 candidates, totalling 500 fits
CPU times: user 1min 13s, sys: 93.8 ms, total: 1min 13s
Wall time: 1min 15s


[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:  1.3min finished


RandomizedSearchCV(cv=StratifiedKFold(n_splits=2, random_state=None, shuffle=False),
          error_score='raise',
          estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
          fit_params={}, iid=True, n_iter=250, n_jobs=1,
          param_distributions={'C': array([  1.00000e-09,   5.62341e-09,   3.16228e-08,   1.77828e-07,
         1.00000e-06,   5.62341e-06,   3.16228e-05,   1.77828e-04,
         1.00000e-03,   5.62341e-03,   3.16228e-02,   1.77828e-01,
         1.00000e+00,   5.62341e+00,   3.16228e+01,   1.77828e+02,
      ...7828e+05,
         1.00000e+06,   5.62341e+06,   3.16228e+07,   1.77828e+08,
         1.00000e+09])},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score=True, scoring='accuracy', verbose=1)

### Best score + params

In [13]:
cv.best_score_, cv.best_params_

(0.98942682248191427, {'C': 1.0, 'gamma': 0.001, 'kernel': 'rbf'})

An example of the "cannonical" cv_results_ table in sklearn:

In [33]:
pd.DataFrame(cv.cv_results_).sort_values("mean_test_score", ascending=False).head()

index  max_test_score  mean_test_score  min_test_score nan_test_score?  \
0        1        0.994435         0.994435        0.994435           False   
254    255        0.994435         0.994435        0.994435           False   
263    264        0.994435         0.994435        0.994435           False   
262    263        0.994435         0.994435        0.994435           False   
261    262        0.994435         0.994435        0.994435           False   

     param_index                                             params  \
0              0  {'C': 177827941.004, 'kernel': 'rbf', 'gamma':...   
254            0  {'C': 177827.941004, 'kernel': 'rbf', 'gamma':...   
263            0  {'C': 177827941.004, 'kernel': 'rbf', 'gamma':...   
262            0  {'C': 1000000.0, 'kernel': 'rbf', 'gamma': 0.001}   
261            0  {'C': 177827941.004, 'kernel': 'rbf', 'gamma':...   

     std_test_score  
0               0.0  
254             0.0  
263             0.0  
262             0.0  
261             0.0

### Using EvolutionaryAlgorithmSearchCV

Again same parameter space, optimize for 10 generations.

**NOTE:** In this version I've changed generations_number to 1 to demonstrate issue 27. It fails to reproduce.

In [23]:
cv = EvolutionaryAlgorithmSearchCV(estimator=SVC(),
                                   params=paramgrid,
                                   scoring="accuracy",
                                   cv=StratifiedKFold(n_splits=2),
                                   verbose=True,
                                   population_size=50,
                                   gene_mutation_prob=0.10,
                                   tournament_size=3,
                                   generations_number=1)
%time cv.fit(X, y)

Types [2, 1, 2] and maxint [24, 0, 24] detected
--- Evolve in 625 possible combinations ---
gen	nevals	avg     	min    	max    	std      
0  	50    	0.911163	0.89872	0.98887	0.0296594
1  	39    	0.934023	0.89872	0.98887	0.0407066
Best individual is: {'kernel': 'rbf', 'C': 31622776.601683792, 'gamma': 0.001}
with fitness: 0.9888703394546466
Wall time: 14.5 s


### Best score + params

In [20]:
cv.best_score_, cv.best_params_

(0.98887033945464664,
 {'C': 5.6234132519034912, 'gamma': 0.001, 'kernel': 'rbf'})

Our cv_results_ table (note, includes all individuals with their mean, max, min, and std test score).

In [24]:
pd.DataFrame(cv.cv_results_).sort_values("mean_test_score", ascending=False).head()

index  max_test_score  mean_test_score  min_test_score nan_test_score?  \
25     76        0.988870         0.988870        0.988870           False   
36     91        0.988870         0.988870        0.988870           False   
9      60        0.988870         0.988870        0.988870           False   
14     65        0.984975         0.984975        0.984975           False   
28     80        0.984975         0.984975        0.984975           False   

    param_index                                             params  \
25            0     {'kernel': 'rbf', 'C': 1000.0, 'gamma': 0.001}   
36            0  {'kernel': 'rbf', 'C': 31622776.6017, 'gamma':...   
9             0  {'kernel': 'rbf', 'C': 31.6227766017, 'gamma':...   
14            0  {'kernel': 'rbf', 'C': 31622.7766017, 'gamma':...   
28            0  {'kernel': 'rbf', 'C': 1000000000.0, 'gamma': ...   

    std_test_score  
25             0.0  
36             0.0  
9              0.0  
14             0.0  
28             0.0